In [117]:
from ipywidgets import interactive
from ollama import Client
from torch.distributed.autograd import context

In [118]:
client = Client(
  host='http://localhost:11434',
  headers={'x-some-header': 'some-value'}
)

In [57]:
from huggingface_hub import notebook_login
notebook_login()

In [58]:
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_pretrained("meta-llama/Llama-3.2-3B")

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = '/Users/jk1/Zotero/storage/NQS7GUXK/Raghu et al. - 2015 - An Official ATSERSJRSALAT Clinical Practice Gui.pdf'

loader = PyPDFLoader(
    pdf_path,
    mode="page",
)
docs = loader.load()
print(len(docs))

17


In [120]:
len(docs[4].page_content)

6750

In [121]:
docs[4].page_content

'guideline were not readdressed in this\nupdate unless the literature search revealed\nnew and pertinent evidence.\nThe committee selected outcomes of\ninterest for each question, using the 2011\ndocument as a guide in addition to\nfollowing the approach suggested by the\nGRADE working group (5, 7). All\noutcomes were identiﬁed ap r i o r i,a n dt h e\ncommittee explicitly rated their relative\nimportance (from the perspective of\na patient with IPF) from not important to\ncritical (7). Ranking outcomes by their\nrelative importance helps focus attention\non those that are most relevant to patients\nand helps resolve or clarify potential\ndisagreements in decision making.\nExamples of critical outcomes include\nmortality or disease progression. Disease\nprogression, deﬁned in the 2011 document\nas increasing respiratory symptoms,\nworsening pulmonary function test (PFT)\nresults, progressive ﬁbrosis on high-\nresolution computed tomography scan,\nacute respiratory decline, or death, ca

In [75]:
def formulate_prompt(guideline_text):
    main_task_formulation = ("Given the following guideline text, give ma a list of all recommendations along with their strength of recommendation and level of confidence in estimates of effect.The exact wording of the recommendations should be respected.\n")
    grade_options_formulation = ("Options for strength of recommendation are:\n- strong recommendation against\n- strong recommendation\n- conditional recommendation\n -conditional recommendation against\n\n")
    level_options_formulation = ("Options for level of confidence in estimates of effect are:\n- high\n- moderate\n- low\n- very low\n\n")
    expected_structure_formulation = ("The output should be in csv formant as such:\n0, recommendation, strength of recommendation, level of confidence in estimates of effect\n1, recommendation, strength of recommendation, level of confidence in estimates of effect\n\n")
    null_handling = "If no recommendation is found in the whole text the output should be 'No recommendation found'.\n\n"
    example_text = ("Example 1:\n Guideline text: 'Guideline on management of ACS. Patients with acute coronary syndrome should be treated with aspirin (strong recommendation, high confidence in estimates of effect). No recommendation can be made regarding the use of ticagrelor.'\n\nExtracted recommendation list:\n0, 'Patients with acute coronary syndrome should be treated with aspirin', 'strong recommendation', 'high'\n'\n\n"
                    "Example 2:\n Guideline text: 'Serum cholesterol and its lipoprotein carriers (LDL, very low-density lipoprotein [VLDL], and HDL) are known to be related to ASCVD. LDL-C is the dominant form of atherogenic cholesterol. VLDL is the chief carrier of triglycerides, and VLDL cholesterol (VLDL-C) is also atherogenic. HDL-C is seemingly not atherogenic. Chylomicrons transport dietary fat; chylomicron atherogenicity is uncertain. The combination of LDL-C and VLDL-C is called non–HDL-C and is more atherogenic than either lipoprotein alone. The main protein embedded in LDL and VLDL is apolipoprotein B (apoB), and like non–HDL-C, apoB is a stronger indicator of atherogenicity than LDL-C alone.'\n\nExtracted recommendation list:\nNo recommendation found.\n\n")
    no_other_text_warning = "No other text should be present in the output.\n\n"
    task = f"{main_task_formulation}{grade_options_formulation}{level_options_formulation}{expected_structure_formulation}{null_handling}{example_text}{no_other_text_warning}"
    prompt = f"{task}Now extract the recommendations from the following text. Guideline text:\n\n{guideline_text}\n\nExtracted recommendation list:\n"
    return prompt

In [76]:
# todo check number of tokens
tokenizer.encode(formulate_prompt(docs[3].page_content))

Encoding(num_tokens=2094, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [128]:
from ollama import Options

response = client.generate(model='deepseek-r1:32b', prompt=formulate_prompt(docs[5].page_content),
                           options=Options(
                temperature=0.0,
                num_ctx=4096,
            ),)
response.response

'<think>\nAlright, I need to extract recommendations from the given guideline text. Let me read through it carefully.\n\nThe text starts by talking about providing suggested interpretations for stakeholders but doesn\'t mention any specific recommendations yet. It then mentions that two questions didn\'t get a recommendation because more evidence was needed. So, those are "no recommendation" cases.\n\nNext, there\'s a section on manuscript preparation and the process of creating the guideline. This part is more about how the guidelines were developed rather than actual recommendations.\n\nThen, under "Recommendations for Specific Treatment Questions," it refers to an online supplement with supporting evidence but doesn\'t list any recommendations here either.\n\nMoving further down, I see Question 1: Should Patients with IPF Be Treated with Anticoagulation? The background and summary of the evidence discuss studies on warfarin use. Then there\'s a clear recommendation: "We recommend th

In [129]:
print(response.response)

<think>
Alright, I need to extract recommendations from the given guideline text. Let me read through it carefully.

The text starts by talking about providing suggested interpretations for stakeholders but doesn't mention any specific recommendations yet. It then mentions that two questions didn't get a recommendation because more evidence was needed. So, those are "no recommendation" cases.

Next, there's a section on manuscript preparation and the process of creating the guideline. This part is more about how the guidelines were developed rather than actual recommendations.

Then, under "Recommendations for Specific Treatment Questions," it refers to an online supplement with supporting evidence but doesn't list any recommendations here either.

Moving further down, I see Question 1: Should Patients with IPF Be Treated with Anticoagulation? The background and summary of the evidence discuss studies on warfarin use. Then there's a clear recommendation: "We recommend that clinicians n

In [130]:
from thefuzz import process
import pandas as pd

path = '/Users/jk1/Downloads/CMCZFLU4_extraction - Sheet1.csv'
recommendation_gt_df = pd.read_csv(path)

process.extractBests(response.response.lower(), recommendation_gt_df.recommendation.str.lower().values,
                                   limit=10)

[('we recommend that clinicians not use warfarin anticoagulation in patients with ipf who do not have a known alternative indication for its use (strong recommendation against, low confidence in estimates of effect).',
  90),
 ('we recommend that clinicians not use ambrisentan, a selective er-a endothelin receptor antagonist, in patients with ipf, regardless of the presence or absence of ph (strong recommendation against, low confidence in estimates of effect).',
  86),
 ('we suggest that clinicians not use bosentan or macitentan, both dual er-a and er-b endothelin receptor antagonists, for the treatment of ipf (conditional recommendation against, low confidence in estimates of effect).',
  86),
 ('we recommend that clinicians not use imatinib in patients with ipf (strong recommendation, moderate confidence in estimates of effect).',
  57),
 ('we recommend that clinicians not use the combination therapy of n-acetylcysteine, azathioprine, and prednisone in patients with ipf (strong reco

In [79]:
# response to csv
import pandas as pd
from io import StringIO

# convert to csv
df = pd.read_csv(StringIO(response.response), header=None)


ParserError: Error tokenizing data. C error: Expected 4 fields in line 2, saw 6


In [115]:
from tqdm import tqdm


def scan_pdf(pdf_path):
    """
    Scan a PDF file for guideline recommendations using a language model.

    Parameters
    ----------
    pdf_path : str
        Path to the PDF file.

    Returns
    -------
    list of all_recommendations per page
    """
    loader = PyPDFLoader(
        pdf_path,
        mode="page",
    )
    docs = loader.load()

    all_recommendations = []

    for page, doc in tqdm(enumerate(docs), total=len(docs)):
        response = client.generate(model='deepseek-r1:32b', prompt=formulate_prompt(doc.page_content),
                                   options=Options(
                    temperature=0.0,
                    num_ctx=4096,
                ),)
        if "no recommendation found" in response.response.lower():
            print(f"No recommendation found in page: {page}")
        else:
            all_recommendations.append(response.response)

    return all_recommendations

In [116]:
all_recommendations = scan_pdf(pdf_path)

 24%|██▎       | 4/17 [21:40<1:09:27, 320.61s/it]

No recommendation found in page: 3


 29%|██▉       | 5/17 [25:50<59:04, 295.40s/it]  

No recommendation found in page: 4


 53%|█████▎    | 9/17 [4:25:10<3:55:42, 1767.83s/it]


KeyboardInterrupt: 

In [106]:
# remove the following
to_remove = ["Here is the extracted recommendation list in CSV format:\n\n", "\nNote: The strength of recommendations and level of confidence in estimates of effect may vary depending on the specific guideline document and the context in which they are used. These recommendations are based on the provided text and may not reflect the full range of recommendations or evidence available for each condition.", "\nNote that some recommendations are conditional or have a low level of confidence in estimates of effect.", "\nNote: The recommendations are based on the provided guideline text."]

recommendations_df = pd.DataFrame()
for i, rec in enumerate(all_recommendations):
    for item in to_remove:
        rec = rec.replace(item, "")
    # convert to csv
    try:
        df = pd.read_csv(StringIO(rec), header=None, quotechar="'", skipinitialspace=True)
    #  print error if notcsv
    except pd.errors.ParserError:
        print(f"Error parsing CSV on page {i}:\n {rec}")
        continue

    df.columns = ['id', 'recommendation', 'class', 'LOE']

    # append
    recommendations_df = pd.concat([recommendations_df, df], ignore_index=True)



In [107]:
recommendations_df

,id,recommendation,class,LOE
0,0,Should Patients with IPF Be Treated with Antic...,conditional recommendation,low
1,1,Should Patients with IPF Be Treated with Imati...,conditional recommendation against,moderate
2,2,Should Patients with IPF Be Treated with Combi...,conditional recommendation,low
3,3,Should Patients with IPF Be Treated with Ambri...,conditional recommendation against,moderate
4,4,Should Patients with IPF Be Treated with Ninte...,strong recommendation,high
...,...,...,...,...
77,5,Lung transplantation for patients with idiopat...,conditional recommendation,moderate
78,6,Bilateral lung transplantation is preferred ov...,strong recommendation,high
79,7,Sildenafil therapy improves walk distance in p...,strong recommendation,high
80,8,Inhaled prostacyclin and iloprost are recommen...,conditional recommendation,moderate


In [108]:
gt_path = '/Users/jk1/Downloads/CMCZFLU4_extraction - Sheet1.csv'

In [126]:
interactive = False
verbose = True

In [127]:
from evaluation_utils import evaluate_guideline_extraction

gt_df = pd.read_csv(gt_path)
accuracy_recommendation, accuracy_grade, accuracy_level, n_missing_recommendations, all_matches_df, missing_recommendations_df = evaluate_guideline_extraction(df, gt_df, interactive=interactive, verbose=verbose)
print(f'Accuracy of recommendations: {accuracy_recommendation:.2f}%')
print(f'Accuracy of grades: {accuracy_grade:.2f}%')
print(f'Accuracy of levels: {accuracy_level:.2f}%')
print(f'Number of missing recommendations: {n_missing_recommendations}')

No match found for "Patients with acute coronary syndrome should be treated with aspirin"
No match found for "Patients with idiopathic pulmonary fibrosis should be treated with bosentan"
No match found for "Patients with idiopathic pulmonary fibrosis should be treated with macitentan"
No match found for "Patients with idiopathic pulmonary fibrosis should receive sildenafil therapy"
No match found for "Patients with idiopathic pulmonary fibrosis should receive acetylcysteine therapy"
No match found for "Lung transplantation for patients with idiopathic pulmonary fibrosis is recommended"
No match found for "Bilateral lung transplantation is preferred over single-lung transplantation for patients with idiopathic pulmonary fibrosis"
No match found for "Sildenafil therapy improves walk distance in patients with idiopathic pulmonary fibrosis"
No match found for "Inhaled prostacyclin and iloprost are recommended for treatment of pulmonary hypertension secondary to lung fibrosis"
No match foun

ZeroDivisionError: division by zero